# Problem 185

### Number Mind

### Solution

The problem is solved using a constraint-problem approach.

First we use a 2D array, where the first axis is equals to the length of the string (16) and second axis is equals to the lenght of the possible digits (10). In each position we can have three values: -1, 0, or 1:

* -1: (digit unassigned) digit at position *i* could have value *j*
* 1: (positive digit assigned) digit at position *i* has value *j*
* 0: (negative digit assigned) digit at position *i* doesn't have value *j*

The algorithm searches in a recursive way (tree depth is equals to 22, number of guesses); at each depth we find for admissible combinations of digits; if that happens:
* Correct (supposed) digits are posed to *1* in position i, j; all other possible values (all other j) ar posed to 0
* We check if the partial solution is coherent with successive guesses: to do that we need to check that the number of positive assigned digits does not exceed the number of guessed digits and that the number of positive assigned digits + the number of unassigned digits is not lower than the number of guessed digits

All these procedures help pruning the search tree. Also, we have a guess with 0 guessed digits; it's useful to use this as the first guess since it will prune some values and won't need backtracking.

I'm able to compute the solution in 1 minute. The solution found has one value unassigned (-1) but is the only possible one.

In [1]:
guesses = [(5616185650518293, 2),
            (3847439647293047, 1),
            (5855462940810587, 3),
            (9742855507068353, 3),
            (4296849643607543, 3),
            (3174248439465858, 1),
            (4513559094146117, 2),
            (7890971548908067, 3),
            (8157356344118483, 1),
            (2615250744386899, 2),
            (8690095851526254, 3),
            (6375711915077050, 1),
            (6913859173121360, 1),
            (6442889055042768, 2),
            (2321386104303845, 0),
            (2326509471271448, 2),
            (5251583379644322, 2),
            (1748270476758276, 3),
            (4895722652190306, 1),
            (3041631117224635, 3),
            (1841236454324589, 3),
            (2659862637316867, 2)]

for i in range(len(guesses)):
    guesses[i] = [int(d) for d in str(guesses[i][0])], guesses[i][1]

In [2]:
guesses = sorted(guesses, key=lambda g: g[1])
guesses.insert(1, guesses.pop())

In [3]:
guesses

[([2, 3, 2, 1, 3, 8, 6, 1, 0, 4, 3, 0, 3, 8, 4, 5], 0),
 ([1, 8, 4, 1, 2, 3, 6, 4, 5, 4, 3, 2, 4, 5, 8, 9], 3),
 ([3, 8, 4, 7, 4, 3, 9, 6, 4, 7, 2, 9, 3, 0, 4, 7], 1),
 ([3, 1, 7, 4, 2, 4, 8, 4, 3, 9, 4, 6, 5, 8, 5, 8], 1),
 ([8, 1, 5, 7, 3, 5, 6, 3, 4, 4, 1, 1, 8, 4, 8, 3], 1),
 ([6, 3, 7, 5, 7, 1, 1, 9, 1, 5, 0, 7, 7, 0, 5, 0], 1),
 ([6, 9, 1, 3, 8, 5, 9, 1, 7, 3, 1, 2, 1, 3, 6, 0], 1),
 ([4, 8, 9, 5, 7, 2, 2, 6, 5, 2, 1, 9, 0, 3, 0, 6], 1),
 ([5, 6, 1, 6, 1, 8, 5, 6, 5, 0, 5, 1, 8, 2, 9, 3], 2),
 ([4, 5, 1, 3, 5, 5, 9, 0, 9, 4, 1, 4, 6, 1, 1, 7], 2),
 ([2, 6, 1, 5, 2, 5, 0, 7, 4, 4, 3, 8, 6, 8, 9, 9], 2),
 ([6, 4, 4, 2, 8, 8, 9, 0, 5, 5, 0, 4, 2, 7, 6, 8], 2),
 ([2, 3, 2, 6, 5, 0, 9, 4, 7, 1, 2, 7, 1, 4, 4, 8], 2),
 ([5, 2, 5, 1, 5, 8, 3, 3, 7, 9, 6, 4, 4, 3, 2, 2], 2),
 ([2, 6, 5, 9, 8, 6, 2, 6, 3, 7, 3, 1, 6, 8, 6, 7], 2),
 ([5, 8, 5, 5, 4, 6, 2, 9, 4, 0, 8, 1, 0, 5, 8, 7], 3),
 ([9, 7, 4, 2, 8, 5, 5, 5, 0, 7, 0, 6, 8, 3, 5, 3], 3),
 ([4, 2, 9, 6, 8, 4, 9, 6, 4, 3, 6, 0, 7, 5, 4, 

In [4]:
import numpy as np
from itertools import combinations
from utils.math import number_from_list_of_digits

comb_k = [list(combinations(list(range(16)), k)) for k in range(4)]

In [5]:
def supposition(values):
    
    l = [str(a) if m == 1 else 'x' for m, a in zip(np.max(values, axis=1), np.argmax(values, axis=1))]
    return "".join(l)

In [6]:
count_depth_0 = 0
max_depth = 0

def rec_search(depth=0, values=None):
        
    global count_depth_0, max_depth
    
    guess, k = guesses[depth]

    for comb in comb_k[k]:
        
        if depth == 1:
            count_depth_0 += 1

        ok_comb = True
        new_values = values.copy()

        for i, v in enumerate(guess):
            if i in comb:
                if new_values[i][v] != 0:
                    new_values[i, :] = 0
                    new_values[i, v] = 1
                else:
                    ok_comb = False
                    break
            else:
                if new_values[i, v] == 1:
                    ok_comb = False
                    break
                else:
                    new_values[i, v] = 0

        if not ok_comb:
            continue
            
        for d in xrange(depth + 1, 22):
            dg, dk = guesses[d]
            unset = 0
            setsum = 0
            
            for i, v in enumerate(dg):
                if new_values[i, v] == -1:
                    unset += 1
                else:
                    setsum += new_values[i, v]
                    
            if setsum > dk or setsum + unset < dk:
                ok_comb = False
                break
                
        if not ok_comb:
            continue

        if depth == 1:
            print count_depth_0, supposition(new_values), max_depth
            
        max_depth = max(max_depth, depth)

        if depth < 21:
            rec_search(depth + 1, new_values)
        else:
            new_values[new_values == -1] = 1
            print new_values
            print number_from_list_of_digits(np.argmax(new_values, axis=1))
            raise ValueError

In [7]:
empty_choices = np.full(shape=(16, 10), fill_value=-1)

try:
    rec_search(values=empty_choices)
except ValueError:
    pass

3 18xx2xxxxxxxxxxx 0
6 18xxxxx4xxxxxxxx 7
10 18xxxxxxxxx2xxxx 10
11 18xxxxxxxxxx4xxx 10
12 18xxxxxxxxxxx5xx 10
13 18xxxxxxxxxxxx8x 10
14 18xxxxxxxxxxxxx9 10
16 1x4x2xxxxxxxxxxx 10
19 1x4xxxx4xxxxxxxx 12
20 1x4xxxxx5xxxxxxx 12
23 1x4xxxxxxxx2xxxx 12
24 1x4xxxxxxxxx4xxx 12
25 1x4xxxxxxxxxx5xx 12
26 1x4xxxxxxxxxxx8x 17
27 1x4xxxxxxxxxxxx9 17
40 1xxx23xxxxxxxxxx 17
43 1xxx2xxx5xxxxxxx 17
46 1xxx2xxxxxx2xxxx 17
47 1xxx2xxxxxxx4xxx 17
48 1xxx2xxxxxxxx5xx 17
49 1xxx2xxxxxxxxx8x 17
50 1xxx2xxxxxxxxxx9 17
52 1xxxx3x4xxxxxxxx 17
53 1xxxx3xx5xxxxxxx 17
56 1xxxx3xxxxx2xxxx 17
57 1xxxx3xxxxxx4xxx 17
58 1xxxx3xxxxxxx5xx 17
59 1xxxx3xxxxxxxx8x 17
60 1xxxx3xxxxxxxxx9 17
70 1xxxxxx45xxxxxxx 17
73 1xxxxxx4xxx2xxxx 17
74 1xxxxxx4xxxx4xxx 17
75 1xxxxxx4xxxxx5xx 17
76 1xxxxxx4xxxxxx8x 17
77 1xxxxxx4xxxxxxx9 17
80 1xxxxxxx5xx2xxxx 17
81 1xxxxxxx5xxx4xxx 17
82 1xxxxxxx5xxxx5xx 17
83 1xxxxxxx5xxxxx8x 17
84 1xxxxxxx5xxxxxx9 17
96 1xxxxxxxxxx24xxx 17
97 1xxxxxxxxxx2x5xx 17
98 1xxxxxxxxxx2xx8x 17
99 1xxxxxxxxxx2